In [1]:
import numpy as np
import pandas as pd
import queue
import matplotlib.pyplot as plt

import sys
backtest_dir = 'C://backtest/backtest/'
if backtest_dir not in sys.path:
    sys.path.insert(0, backtest_dir)
    
from Backtest import *
from BayesianOptimization import *
from ADX_BOLLStrategy import ADX_BOLLStrategy
from Backtest.open_json_gz_files import open_json_gz_files
from Backtest.generate_bars import generate_bars

In [3]:
def run_backtest(config, trading_data, ohlc_data, window_ADX, window_BOLL, a):
    config['title'] = "ADX_BOLLStrategy" + "_" + str(window_ADX) + "_" + str(window_BOLL) + "_" + str(a)
    print("---------------------------------")
    print(config['title'])
    print("---------------------------------")
    
    events_queue = queue.Queue()

    data_handler = OHLCDataHandler(
        config, events_queue,
        trading_data = trading_data, ohlc_data = ohlc_data
    )
    strategy = ADX_BOLLStrategy(config, events_queue, data_handler,
                           window_ADX = window_ADX,
                            window_BOLL = window_BOLL, a = a)

    backtest = Backtest(config, events_queue, strategy,
                        data_handler= data_handler)

    results = backtest.start_trading()
    
    if results['trade_info']['trading_num'] > 30 and results['trade_info']['win_pct'] > 0.3:
        return (results['cum_returns'][-1] - 1)
    else:
        return (-1)

## Setting

In [4]:
config = {
    "csv_dir": "C:/backtest/Binance",
    "out_dir": "C:/backtest/results/BOLLStrategy",
    "title": "BOLLStrategy",
    "is_plot": False,
    "save_plot": False,
    "save_tradelog": False,
    "start_date": pd.Timestamp("2017-04-01T00:0:00", freq="60" + "T"),  # str(freq) + "T"
    "end_date": pd.Timestamp("2018-09-01T00:00:00", freq="60" + "T"),
    "equity": 1.0,
    "freq": 60,  # min
    "commission_ratio": 0.001,
    "suggested_quantity": None,     # None or a value
    "max_quantity": None,           # None or a value, Maximum purchase quantity
    "min_quantity": None,           # None or a value, Minimum purchase quantity
    "min_handheld_cash": None,      # None or a value, Minimum handheld funds
    "exchange": "Binance",
    "tickers": ['BTCUSDT']
}

In [5]:
# trading_data = {}
# for ticker in config['tickers']:
#     # trading_data[ticker] = open_gz_files(config['csv_dir'], ticker)
#     trading_data[ticker] = pd.read_hdf(config['csv_dir'] + '\\' + ticker + '.h5', key=ticker)

ohlc_data = {}
for ticker in config['tickers']:
    # ohlc_data[ticker] = generate_bars(trading_data, ticker, config['freq'])
    ohlc_data[ticker] = pd.read_hdf(config['csv_dir'] + '\\' + ticker +'_OHLC_60min.h5', key=ticker)

trading_data = None

In [6]:
gp_params = {"alpha": 1e-5}

## Acquisition Function "Upper Confidence Bound"

* Prefer exploitation (kappa=1.0)

* Prefer exploration (kappa=10)


## Acquisition Function "Expected Improvement"

* Prefer exploitation (xi=1e-4)
* Prefer exploration (xi=0.1)

In [7]:
BO = BayesianOptimization(
    run_backtest,
    {'a': (0, 0.25),
    'b': (0, 0.2)},
    is_int = [0, 0], 
    invariant = {
        'config': config,
        'trading_data': trading_data,
        'ohlc_data': ohlc_data
    },
    random_state = 1
)
# BO.explore({
#     'short_window': np.arange(1, 120, 20),
#     'delta_window': np.arange(1, 120, 20)
#     },
#     eager=True)
BO.maximize(init_points=40, n_iter=60, acq="ei", xi=0.01, **gp_params)

Initialization
-----------------------------------------------------
 Step |   Time |      Value |         a |         b | 
---------------------------------
BOLLStrategy_0.1042555011756435_0.19777221778129894
---------------------------------
Data Time Interval for BTCUSDT:
	Start Date	: 2017-10-27 22:00:00
	End Date	: 2018-08-07 23:00:00
---------------------------------
Running Backtest...
---------------------------------
---------------------------------
Backtest complete.
---------------------------------
Sharpe Ratio: -0.1716719087
Sortino Ratio: -0.2432502951
Information Ratio: -1.4300210524
Max Drawdown: 0.7482050821
Max Drawdown Duration: 560400
Total Returns: -0.3248919092
Annualized Returns: -0.4174934905
Compound Annual Growth Rate: -0.3964116798
---------------------------------
Trades: 7
Trade Winning: 28.57%
Average Trade: 2.52%
Average Win: 54.09%
Average Loss: -18.11%
Best Trade: 103.19%
Worst Trade: -20.54%
Worst Trade Date: 8 days 20:00:00
Avg Days in Trade: 15 days

---------------------------------
Backtest complete.
---------------------------------
Sharpe Ratio: 1.1816784574
Sortino Ratio: 1.3404566855
Information Ratio: 0.1705147336
Max Drawdown: 0.5881710423
Max Drawdown Duration: 560400
Total Returns: 0.5851598962
Annualized Returns: 0.7519437716
Compound Annual Growth Rate: 0.8075838492
---------------------------------
Trades: 14
Trade Winning: 42.86%
Average Trade: 7.18%
Average Win: 28.34%
Average Loss: -8.69%
Best Trade: 126.89%
Worst Trade: -16.68%
Worst Trade Date: 1 days 09:00:00
Avg Days in Trade: 9 days 02:12:51
---------------------------------
    9 | 00m38s |   -1.00000 | 0.099192 | 0.057555 | 
---------------------------------
BOLLStrategy_0.13470418350083924_0.026005714423655537
---------------------------------
Data Time Interval for BTCUSDT:
	Start Date	: 2017-10-27 22:00:00
	End Date	: 2018-08-07 23:00:00
---------------------------------
Running Backtest...
---------------------------------
--------------------------------

---------------------------------
Backtest complete.
---------------------------------
Sharpe Ratio: 1.4264385433
Sortino Ratio: 1.4728471852
Information Ratio: 0.3506872230
Max Drawdown: 0.6115228015
Max Drawdown Duration: 560400
Total Returns: 0.7956120854
Annualized Returns: 1.0223796197
Compound Annual Growth Rate: 1.1216261908
---------------------------------
Trades: 14
Trade Winning: 35.71%
Average Trade: 8.12%
Average Win: 36.08%
Average Loss: -7.42%
Best Trade: 125.77%
Worst Trade: -16.63%
Worst Trade Date: 1 days 07:00:00
Avg Days in Trade: 8 days 00:25:43
---------------------------------
   18 | 00m33s |   -1.00000 | 0.139672 | 0.029346 | 
---------------------------------
BOLLStrategy_0.03509673464880844_0.11786110738065686
---------------------------------
Data Time Interval for BTCUSDT:
	Start Date	: 2017-10-27 22:00:00
	End Date	: 2018-08-07 23:00:00
---------------------------------
Running Backtest...
---------------------------------
---------------------------------

---------------------------------
Backtest complete.
---------------------------------
Sharpe Ratio: 0.1337458628
Sortino Ratio: 0.1941460990
Information Ratio: -1.0013798328
Max Drawdown: 0.7394405655
Max Drawdown Duration: 560400
Total Returns: -0.1588237747
Annualized Returns: -0.2040921617
Compound Annual Growth Rate: -0.1992847986
---------------------------------
Trades: 15
Trade Winning: 26.67%
Average Trade: 2.59%
Average Win: 38.73%
Average Loss: -10.55%
Best Trade: 121.13%
Worst Trade: -20.34%
Worst Trade Date: 4 days 15:00:00
Avg Days in Trade: 13 days 14:52:00
---------------------------------
   27 | 00m09s |   -1.00000 | 0.021261 | 0.132759 | 
---------------------------------
BOLLStrategy_0.00976369580822059_0.10297782241166172
---------------------------------
Data Time Interval for BTCUSDT:
	Start Date	: 2017-10-27 22:00:00
	End Date	: 2018-08-07 23:00:00
---------------------------------
Running Backtest...
---------------------------------
---------------------------

---------------------------------
Running Backtest...
---------------------------------
---------------------------------
Backtest complete.
---------------------------------
Sharpe Ratio: 1.5178261100
Sortino Ratio: 1.7187854000
Information Ratio: 0.4987917814
Max Drawdown: 0.5501738646
Max Drawdown Duration: 560400
Total Returns: 0.9142026935
Annualized Returns: 1.1747712476
Compound Annual Growth Rate: 1.3033550784
---------------------------------
Trades: 17
Trade Winning: 35.29%
Average Trade: 7.26%
Average Win: 31.97%
Average Loss: -6.23%
Best Trade: 130.80%
Worst Trade: -15.39%
Worst Trade Date: 1 days 19:00:00
Avg Days in Trade: 7 days 22:35:18
---------------------------------
   36 | 00m08s |   -1.00000 | 0.078879 | 0.033071 | 
---------------------------------
BOLLStrategy_0.17162523192039592_0.1855017160792068
---------------------------------
Data Time Interval for BTCUSDT:
	Start Date	: 2017-10-27 22:00:00
	End Date	: 2018-08-07 23:00:00
---------------------------------


KeyboardInterrupt: 

In [ ]:
print(BO.res['max'])

In [ ]:
BO.maximize(init_points=0, n_iter=60, acq="ei", xi=0.0001, **gp_params)

In [ ]:
print(BO.res['max'])

In [ ]:
Target = pd.DataFrame({'Parameters': BO.X.tolist(), 'Target': BO.Y})
# Target.to_csv(config['out_dir'] + "/target_ei.csv")
Target.sort_values(by = "Target")

## Acquisition Function "Probability of Improvement"

* Prefer exploitation (xi=1e-4)
* Prefer exploration (xi=0.1)